In [7]:
import requests
import json
import datetime
import time
import pygal
from bs4 import BeautifulSoup

class mobile01:
    
    def __init__(self):
        
        self.domain_name = 'www.mobile01.com'
        self.path = {
            'ASUS':'topiclist.php?sort=topictime&f=588&p=', 
            'ACER':'topiclist.php?sort=topictime&f=564&p=',
            'HTC':'topiclist.php?sort=topictime&f=566&p=',
            'SAMSUNG':'topiclist.php?sort=topictime&f=568&p='
        }
        self.page = 1
        self.begin = datetime.date(2016, 7, 17)
        self.start_date = self.begin
        self.begin = self.begin.timetuple()
        self.begin = time.mktime(self.begin)
        self.end = datetime.date(2016, 7, 24)
        self.end_date = self.end
        self.end = self.end.timetuple()
        self.end = time.mktime(self.end)
        self.day_count = (self.end_date - self.start_date).days + 1
        self.schema = {}
        
        for brand in self.path:
            
            if brand not in self.schema:
                self.schema[brand] = {}
                
            for single_date in (self.start_date + datetime.timedelta(n) for n in range(self.day_count)):
                self.schema[brand][str(single_date)] = 0
        
    def get_forum_list(self):
        
        count = self.schema
        
        for brand in self.path:
            
            self.page = 1
            out = False
                
            while False == out:
                
                url = 'http://' + self.domain_name + '/' + self.path[brand] + str(self.page)
                rs = requests.get(url)
                forum_list = BeautifulSoup(rs.text, 'lxml')

                for thread in forum_list.select('table[summary=文章列表] tr'):
                    
                    if 0 == len(thread.select('span.redpoint')) and 0 == len(thread.select('th')):
                        
                        authur_date = thread.select('td.authur p')[0].getText().split(' ')
                        authur_date = authur_date[0]
                        key = authur_date
                        authur_date = datetime.datetime.strptime(authur_date, '%Y-%m-%d').timetuple()
                        timestamp = time.mktime(authur_date)
                        
                        if self.begin <= timestamp and self.end >= timestamp:
                                count[brand][key] = count[brand][key] + 1
                        else:
                            out = True
                            
                self.page = self.page + 1
        
        self.schema = count
        
    def draw_graph(self):
        
        count = {};
        
        for brand in self.path:
            
            if brand not in count:
                count[brand] = self.schema[brand].values()
        
        line_chart = pygal.Bar()
        line_chart.title = '手機品牌討論熱度評比 (依發文數)'
        line_chart.x_labels = map(str, (self.start_date + datetime.timedelta(n) for n in range(self.day_count)))
        
        for brand in count:
            line_chart.add(brand, count[brand])
            
        line_chart.render_to_file('rating.svg')
        

In [8]:
m01 = mobile01()
m01.get_forum_list()
m01.draw_graph()